In [7]:
import os
import sys
import cv2
import numpy as np
import json
from PIL import Image, ImageDraw, ImageFont
import math
import boto3

def draw_ocr_box_txt(image,
                     boxes,
                     txts,
                     scores=None,
                     drop_score=0.5,
                     font_path="./doc/simfang.ttf"):
    h, w = image.height, image.width
    img_left = image.copy()
    img_right = Image.new('RGB', (w, h), (255, 255, 255))

    import random

    random.seed(0)
    draw_left = ImageDraw.Draw(img_left)
    draw_right = ImageDraw.Draw(img_right)
    for idx, (box, txt) in enumerate(zip(boxes, txts)):
        if scores is not None and scores[idx] < drop_score:
            continue
        color = (random.randint(0, 255), random.randint(0, 255),
                 random.randint(0, 255))
        draw_left.polygon([box[0][0], box[0][1], box[1][0], box[1][1], box[2][0], box[2][1], box[3][0], box[3][1]], fill=color)
        draw_right.polygon(
            [
                box[0][0], box[0][1], box[1][0], box[1][1], box[2][0],
                box[2][1], box[3][0], box[3][1]
            ],
            outline=color)
        box_height = math.sqrt((box[0][0] - box[3][0])**2 + (box[0][1] - box[3][
            1])**2)
        box_width = math.sqrt((box[0][0] - box[1][0])**2 + (box[0][1] - box[1][
            1])**2)
        if box_height > 2 * box_width:
            font_size = max(int(box_width * 0.9), 10)
            font = ImageFont.truetype(font_path, font_size, encoding="utf-8")
            cur_y = box[0][1]
            for c in txt:
                char_size = font.getsize(c)
                draw_right.text(
                    (box[0][0] + 3, cur_y), c, fill=(0, 0, 0), font=font)
                cur_y += char_size[1]
        else:
            font_size = max(int(box_height * 0.8), 10)
            font = ImageFont.truetype(font_path, font_size, encoding="utf-8")
            draw_right.text(
                [box[0][0], box[0][1]], txt, fill=(0, 0, 0), font=font)
    img_left = Image.blend(image, img_left, 0.5)
    img_show = Image.new('RGB', (w * 2, h), (255, 255, 255))
    img_show.paste(img_left, (0, 0, w, h))
    img_show.paste(img_right, (w, 0, w * 2, h))
    return np.array(img_show)

In [10]:
def download_and_draw(image_uri, response):

    s3_client = boto3.client('s3')

    bucket = 'sagemaker-cn-north-1-813110655017'

    image_file = image_uri.split('/')[-1]
    try:
        s3_client.download_file(bucket, image_uri, image_file)
    except:
        image_file = '1.jpg'



    body = json.loads(response["body"])


    drop_score = 0.5
    font_path = "simfang.ttf"
    img = cv2.imread(image_file)

    image = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    boxes = body["bbox"]
    txts = body["label"]
    scores = body["confidences"]
    # print(boxes)
    # print(txts)
    # print(scores)

    draw_img = draw_ocr_box_txt(
                    image,
                    boxes,
                    txts,
                    scores,
                    drop_score=drop_score,
                    font_path=font_path)

    draw_img_save = "./"
    if not os.path.exists(draw_img_save):
        os.makedirs(draw_img_save)
    image_file = image_file[:-4] + "_result.png" 
    cv2.imwrite(
        os.path.join(draw_img_save, os.path.basename(image_file)),
        draw_img[:, :, ::-1])
    return image_file

In [11]:
image_uri = "paddleocr/3.PNG"
response = {
  "statusCode": 200,
  "body": "{\"label\": [\"4136 HOI\", \"o 0uter Ring Tuune\", \"口\", \"6km/h\", \"团道\", \"Ramp\", \"milm\", \"st Realse F39902060 2021 03 26 13 43 5701\", \"0\", \"0:00\"], \"confidences\": [0.725160539150238, 0.8302705883979797, 0.6408441662788391, 0.9979934692382812, 0.5427997708320618, 0.9952331185340881, 0.6441168785095215, 0.9549294710159302, 0.9993733763694763, 0.9576385021209717], \"bbox\": [[[658.0, 2.0], [712.0, 2.0], [712.0, 15.0], [658.0, 15.0]], [[757.0, 3.0], [870.0, 7.0], [870.0, 27.0], [757.0, 23.0]], [[1725.0, 16.0], [1744.0, 16.0], [1744.0, 32.0], [1725.0, 32.0]], [[1.0, 29.0], [81.0, 29.0], [81.0, 55.0], [1.0, 55.0]], [[967.0, 159.0], [1018.0, 162.0], [1016.0, 187.0], [966.0, 185.0]], [[974.0, 193.0], [1011.0, 193.0], [1011.0, 204.0], [974.0, 204.0]], [[302.0, 278.0], [371.0, 278.0], [371.0, 295.0], [302.0, 295.0]], [[7.0, 749.0], [1012.0, 747.0], [1012.0, 789.0], [7.0, 791.0]], [[1.0, 901.0], [14.0, 901.0], [14.0, 917.0], [1.0, 917.0]], [[1745.0, 899.0], [1778.0, 899.0], [1778.0, 918.0], [1745.0, 918.0]]], \"shape\": [1013, 1780, 3]}"
}

result = download_and_draw(image_uri, response)
print(result)

3_result.png
